In [8]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

In [9]:
df = pd.read_csv('outputs/dataset_20210523.csv')
df_champions = pd.read_csv('data/dataset_champions.csv')
df

,game,blueTop,blueTopGP,blueTopWR,blueTopKDA,blueJungle,blueJungleGP,blueJungleWR,blueJungleKDA,blueMid,...,redMidKDA,redAdc,redAdcGP,redAdcWR,redAdcKDA,redSupport,redSupportGP,redSupportWR,redSupportKDA,result
0,ESPORTSTMNT03/1632489,82,8,0.50,2.9,104,27,0.85,5.9,142,...,3.6,81,21,0.57,4.3,89,10,0.60,2.5,0
1,ESPORTSTMNT03/1632500,79,14,0.36,2.6,104,63,0.63,5.1,68,...,4.9,21,10,0.70,5.3,3,11,0.55,2.8,1
2,ESPORTSTMNT03/1632502,79,14,0.36,2.6,104,63,0.63,5.1,142,...,4.4,21,10,0.70,5.3,3,11,0.55,2.8,1
3,6909-9183,43,2,0.00,0.9,80,5,0.60,3.9,134,...,5.2,145,43,0.60,6.3,3,4,0.75,4.5,1
4,6909-9184,24,16,0.56,2.8,120,11,0.55,3.6,61,...,5.2,145,43,0.60,6.3,57,2,0.50,1.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3824,7336-9851,14,9,0.56,4.9,120,9,0.44,4.3,61,...,4.7,429,17,0.12,3.6,111,9,0.22,1.3,0
3825,ESPORTSTMNT01/1886793,164,8,0.50,2.1,77,6,0.83,4.3,112,...,2.0,429,45,0.71,6.5,412,20,0.55,4.5,0
3826,7155-9853,14,2,0.50,8.0,77,13,0.46,5.7,142,...,5.1,145,14,0.57,3.8,79,3,0.67,1.6,0
3827,7154-9854,58,32,0.38,2.6,77,7,0.29,3.1,112,...,7.3,523,32,0.44,3.1,412,21,0.57,3.3,0


In [10]:
y = df['result'].copy()
X = df.drop(['blueTop', 'blueJungle', 'blueMid', 'blueADC', 'blueSupport', 'redTop', 'redJungle', 'redMid', 'redAdc', 'redSupport', 'result', 'game'],axis=1)
X

,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,blueMidWR,blueMidKDA,blueADCGP,...,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
0,8,0.50,2.9,27,0.85,5.9,59,0.76,10.7,10,...,4.3,13,0.46,3.6,21,0.57,4.3,10,0.60,2.5
1,14,0.36,2.6,63,0.63,5.1,1,0.00,1.0,20,...,4.1,55,0.69,4.9,10,0.70,5.3,11,0.55,2.8
2,14,0.36,2.6,63,0.63,5.1,5,0.40,2.9,2,...,6.4,3,0.67,4.4,10,0.70,5.3,11,0.55,2.8
3,2,0.00,0.9,5,0.60,3.9,36,0.75,6.2,5,...,3.0,6,0.67,5.2,43,0.60,6.3,4,0.75,4.5
4,16,0.56,2.8,11,0.55,3.6,50,0.64,6.1,5,...,2.9,6,0.67,5.2,43,0.60,6.3,2,0.50,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3824,9,0.56,4.9,9,0.44,4.3,31,0.61,6.0,28,...,4.2,35,0.54,4.7,17,0.12,3.6,9,0.22,1.3
3825,8,0.50,2.1,6,0.83,4.3,8,0.38,4.5,75,...,2.3,5,0.20,2.0,45,0.71,6.5,20,0.55,4.5
3826,2,0.50,8.0,13,0.46,5.7,19,0.53,8.1,10,...,4.5,9,0.78,5.1,14,0.57,3.8,3,0.67,1.6
3827,32,0.38,2.6,7,0.29,3.1,9,0.44,4.2,5,...,2.7,2,0.50,7.3,32,0.44,3.1,21,0.57,3.3


In [11]:
def preprocess_input(X,y):
    skf = StratifiedKFold(n_splits=10)
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained
Support Vector Machine (Linear Kernel) trained
C:\Users\linco\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [13]:
scores_list = []
for name,model in models.items():
    scores_list.append({
    'Model': name,
    'Accuracy': accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [14]:
scores

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.924282,0.923729,0.913408,0.918539
1,Support Vector Machine (Linear Kernel),0.925587,0.923944,0.916201,0.920056
2,Support Vector Machine (RBF Kernel),0.930809,0.927171,0.924581,0.925874
3,Decission Tree,0.886423,0.871233,0.888268,0.879668
4,Adaboost,0.939948,0.928571,0.944134,0.936288
5,Random Forest,0.939948,0.950867,0.918994,0.934659
6,Gradient Boosting Classifier,0.949086,0.962319,0.927374,0.944523
